<a href="https://colab.research.google.com/github/alarcon7a/impulse-rag/blob/main/Template_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install fairseq2
!pip install git+https://github.com/huggingface/transformers.git sentencepiece
!pip install -U transformers
!pip install -q llama_index openai gradio chromadb

## Importar librerias

In [ ]:
from llama_index.llms import OpenAI, Gemini
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index import ServiceContext
import gradio as gr
import openai
import requests
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.response.notebook_utils import display_response
from llama_index.prompts import PromptTemplate
import os
from getpass import getpass
import scipy


## Cargar variables

In [ ]:
GOOGLE_API_KEY = getpass('Enter the secret value for Google Key: ')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

Enter the secret value for Google Key: ··········


In [ ]:
OPENAI_API_KEY = getpass('Enter the secret value for OpenAI Key: ')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Enter the secret value for OpenAI Key: ··········


## Uso de LLM

### OpenAI

### Google Gemini

# Aplicando RAG!!!

### Algo de customización

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L12-v2")

### Cambiando el template

In [ ]:
template = (
    "Dado el contexto que te proporcionare responde las preguntas \n"
    "Contexto:\n"
    "################################\n"
    "{context_str}"
    "################################\n"
    "Responde en español: {query_str}\n"
)
qa_template = PromptTemplate(template)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

## Como ocurre esto?

## Apliquemos algunas mejoras

In [ ]:
template = (
    "Dado el contexto que te proporcionare responde las preguntas \n"
    "Contexto:\n"
    "################################\n"
    "{context_str}"
    "################################\n"
    "Responde en español: {query_str}\n"
)
qa_template = PromptTemplate(template)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

## Aplicando aun mas mejoras !!

In [ ]:
from transformers import AutoProcessor, SeamlessM4Tv2Model
import torchaudio
import torch

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    dtype = torch.float16
else:
    device = torch.device("cpu")
    dtype = torch.float32


In [ ]:
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
model = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large").to(device)

preprocessor_config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/19.7k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.17M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/211k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.24G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/9.91M [00:00<?, ?B/s]

In [ ]:
AUDIO_SAMPLE_RATE = model.config.sampling_rate ## 16000
MAX_INPUT_AUDIO_LENGTH = 60

In [ ]:
## Permite ejecutar el chatbot en langchain creado previamente, retorna la ultima respuesta del chatbot y el historico de la conversación
def generate_llm_response(input_message, messages):
    response = query_engine.query(input_message)
    messages.extend(['User: ' + input_message, 'IA: ' + response.response])
    chat_transcription = "\n ".join(messages)
    return chat_transcription, response.response


In [ ]:
## Recibe el audio directamente del microfono y lo procesa a la forma manejada por Seamless para el control de audio (audio_inputs)
def preprocess_audio(input_audio: str):
    arr, org_sr = torchaudio.load(input_audio)
    new_arr = torchaudio.functional.resample(arr, orig_freq=org_sr, new_freq=AUDIO_SAMPLE_RATE)
    max_length = int(MAX_INPUT_AUDIO_LENGTH * AUDIO_SAMPLE_RATE)
    if new_arr.shape[1] > max_length:
        new_arr = new_arr[:, :max_length]
        gr.Warning(f"Input audio is too long. Only the first {MAX_INPUT_AUDIO_LENGTH} seconds is used.")
    audio_inputs = processor(audios=new_arr, return_tensors="pt").to(device)
    return audio_inputs

In [ ]:
## Recibe el tensor(audio_inputs) para pasarlo a un texto en ingles, sin importar el lenguaje de origen
def speech_to_text(audio_inputs):
    output_tokens = model.generate(**audio_inputs, tgt_lang='spa', generate_speech=False)
    translated_text_from_audio = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
    print(translated_text_from_audio)
    return translated_text_from_audio

In [ ]:
## Recibe la respuesta en texto del LLM para convertir la respuesta a audio en español e ingles y guardar los dos archivos .wav de salida
def text_to_speech(text_input):
    text_inputs = processor(text = text_input, src_lang=["spa"], return_tensors="pt").to(device)

    spanish_audio = model.generate(**text_inputs, tgt_lang="spa")[0].cpu().numpy().squeeze()

    scipy.io.wavfile.write("spanish_audio.wav", rate=AUDIO_SAMPLE_RATE, data=spanish_audio)

    return './spanish_audio.wav'

In [ ]:
### Ahora dejemos todo junto en una unica funcion que use las anteriores
messages = []

def all_together(audio):
    global messages
    arr_audio = preprocess_audio(audio)
    query_input = speech_to_text(arr_audio)
    llm_response, last_response = generate_llm_response( query_input,messages)
    spanish_output = text_to_speech(last_response)

    return llm_response, spanish_output

## Gradio

In [ ]:
iface = gr.Interface(
    fn=all_together,
    inputs=
        gr.Audio(type="filepath"),
    outputs=[
        gr.Textbox(label="Chat"),
        gr.Audio(label="Audio en Español", autoplay=False)],
    title="RAG con Audio",
    description="Graba tu voz usando el micrófono"
)

iface.launch(debug=True)


In [ ]:
iface.close()

Closing server running on port: 7860
